In [ ]:
import os
import pandas as pd
import numpy as np

# pick original dataset
is_celeba = False

if is_celeba:
    dataset = "aug_precise_prompts_strong"
    folder = "../CelebA/"
    
else:
    dataset = "mixed_caps_total_70k"
    folder = "../fairface/dataset/augmentations/"

Build operations.csv for main.py

In [2]:
dir_path = folder + dataset

filenames = [os.path.join(dataset, filename) for filename in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, filename))]
df = pd.DataFrame({'src': filenames})
df.to_csv('operations_70k.csv', index=False)

Calculate fidelity scores from logits

In [3]:
# Load CSV file into a Pandas DataFrame
df = pd.read_csv(f'results_tst/{dataset}/{dataset}.csv')

# Calculate the fidelity using sigmoid, shifting and clipping
correction_factor = 3.
sigmoid_values = 1 / (1 + np.exp(-df['Grag2021_latent'] / correction_factor))
shifted_sig_values = 2 * (sigmoid_values - 0.5)
shifted_sig_values[shifted_sig_values < 0] = 0
fidelity = shifted_sig_values

# Remove everything that comes before the last "/" in the first column
df['src'] = df['src'].str.split('/').str[-1]

# Create a new DataFrame with the modified column names and sigmoid values
result_df = pd.DataFrame({'Image_Name': df['src'], 'Fidelity_Score': fidelity})

# Save the new DataFrame to a CSV file
result_df.to_csv(f'fidelities_{dataset}.csv', index=False)


Merge FairFace and DM_Image_Detection fidelity classification score

In [6]:
ff_fid = pd.read_csv("../fairface/outputs/train_aug_10k_precise_predictions.csv")
synth_fid = pd.read_csv("fidelities_10k_precise_prompts.csv")
merged = pd.merge(ff_fid, synth_fid)

# # harmonic mean
merged["Total_Fidelity"] = 2 * (merged["Race_Score_Goal"] * merged["Fidelity_Score"]) / (merged["Race_Score_Goal"] + merged["Fidelity_Score"])

# Fidelity twice as important
# merged["Total_Fidelity"] = (2 * merged["Fidelity_Score"] + merged["Race_Score_Goal"]) / 3

# merged["Total_Fidelity"] = merged["Fidelity_Score"]

print(merged.head())
merged.to_csv("total_fidelities_10k_harm_precise_prompts.csv")


                     Image_Name    Race  Gender  Race_Score_Goal  \
0            000001_0_black.jpg   Black       1            0.953   
1           000001_1_indian.jpg  Latino       1            0.207   
2           000001_2_latino.jpg  Latino       1            0.628   
3   000001_3_middle eastern.jpg  Latino       1            0.019   
4  000001_4_southeast asian.jpg  Latino       1            0.118   

                                   Race_Scores Gender_Scores  Fidelity_Score  \
0  [0.953 0.004 0.041 0.    0.    0.    0.002]       [0. 1.]        0.987559   
1  [0.004 0.207 0.642 0.041 0.001 0.    0.105]       [0. 1.]        0.979519   
2  [0.009 0.003 0.628 0.009 0.001 0.    0.35 ]       [0. 1.]        0.978242   
3  [0.005 0.007 0.546 0.019 0.    0.    0.422]       [0. 1.]        0.975398   
4  [0.004 0.005 0.717 0.001 0.118 0.042 0.112]       [0. 1.]        0.949471   

   Total_Fidelity  
0        0.969972  
1        0.341774  
2        0.764936  
3        0.037274  
4        0